# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import numpy as np # library to handle data in a vectorized manner
import requests
from bs4 import BeautifulSoup
import pandas as pd # library for data analsysis
import json
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Scraping from Wiki page and putting to data frame

In [2]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df_clean = df[df['Borough'] != 'Not assigned']
df_clean

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


## Getting latitude and the longitude coordinates of each neighborhood.

In [5]:
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format('M8Z'))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

#### Note:  the geocoder library seemed to hang forever, so I used the CSV instead

In [6]:
lat_lng_df = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lng_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [7]:
# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [8]:
for index, row in df_clean.iterrows():
    post_code = row['Postcode'] 
    borough = row['Borough'] 
    neighborhood_name = row['Neighbourhood']
    
    lat_long = lat_lng_df.loc[lat_lng_df['Postal Code'] == post_code]

    if lat_long.empty == False:
        neighborhood_lat = lat_long['Latitude'].values[0]
        neighborhood_lon = lat_long['Longitude'].values[0]
    else:
        neighborhood_lat = 'Not Found'
        neighborhood_lon = 'Not Found'
    
    neighborhoods = neighborhoods.append({'PostalCode': post_code,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Not assigned,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


## Using geopy library to get the latitude and longitude values of Toronto

In [10]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Connecting to Foursquare and exploring

In [12]:
CLIENT_ID = '2RTVW4MQWUB4HHSG5BJUIBNXO4EDACHOXJ1GQ5PWOJ0VVBDT' # your Foursquare ID
CLIENT_SECRET = 'IOVG5OQTNEBTVUB0ZGVUVQOKXU41VXAVWXKTQRHMDSALU1TL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2RTVW4MQWUB4HHSG5BJUIBNXO4EDACHOXJ1GQ5PWOJ0VVBDT
CLIENT_SECRET:IOVG5OQTNEBTVUB0ZGVUVQOKXU41VXAVWXKTQRHMDSALU1TL


In [13]:
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=2RTVW4MQWUB4HHSG5BJUIBNXO4EDACHOXJ1GQ5PWOJ0VVBDT&client_secret=IOVG5OQTNEBTVUB0ZGVUVQOKXU41VXAVWXKTQRHMDSALU1TL&v=20180605&ll=43.653963,-79.387207&radius=500&limit=100'

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d3b0eb2fd16bb0039d42a34'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 77,
  'suggestedBounds': {'ne': {'lat': 43.6584630045, 'lng': -79.38099903084075},
   'sw': {'lat': 43.649462995499995, 'lng': -79.39341496915925}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Japango,Sushi Restaurant,43.655268,-79.385165
2,Cafe Plenty,Café,43.654571,-79.389450
3,Textile Museum of Canada,Art Museum,43.654396,-79.386500
4,Poke Guys,Poke Place,43.654895,-79.385052


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

77 venues were returned by Foursquare.


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
       
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )



Parkwoods
Victoria Village
Harbourfront
Regent Park
Lawrence Heights
Lawrence Manor
Not assigned
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design E

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(4441, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
3,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [21]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
Agincourt North,3,3,3,3,3,3
Albion Gardens,9,9,9,9,9,9
Alderwood,9,9,9,9,9,9
Bathurst Manor,19,19,19,19,19,19
Bathurst Quay,16,16,16,16,16,16
Bayview Village,4,4,4,4,4,4
Beaumond Heights,9,9,9,9,9,9


In [22]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 279 uniques categories.


In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_onehot.shape

(4441, 279)

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.0,0.000000,0.0
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
2,Agincourt North,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
3,Albion Gardens,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
4,Alderwood,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
5,Bathurst Manor,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.0,0.000000,0.0
6,Bathurst Quay,0.000000,0.0,0.000000,0.062500,0.0625,0.0625,0.125,0.1875,0.125,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
7,Bayview Village,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
8,Beaumond Heights,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
9,Bedford Park,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0


In [26]:
toronto_grouped.shape

(207, 279)

#### Let's print each neighborhood along with the top 5 most common venues

In [27]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Agincourt----
            venue  freq
0  Breakfast Spot  0.25
1          Lounge  0.25
2  Clothing Store  0.25
3    Skating Rink  0.25
4     Yoga Studio  0.00


----Agincourt North----
                venue  freq
0          Playground  0.33
1                 Gym  0.33
2                Park  0.33
3         Yoga Studio  0.00
4  Miscellaneous Shop  0.00


----Albion Gardens----
                 venue  freq
0        Grocery Store  0.22
1          Pizza Place  0.11
2          Coffee Shop  0.11
3           Beer Store  0.11
4  Fried Chicken Joint  0.11


----Alderwood----
            venue  freq
0     Pizza Place  0.22
1            Pool  0.11
2    Skating Rink  0.11
3  Sandwich Place  0.11
4             Pub  0.11


----Bathurst Manor----
                       venue  freq
0                Coffee Shop  0.11
1                

                       venue  freq
0                Coffee Shop  0.11
1                 Restaurant  0.05
2       Fast Food Restaurant  0.05
3              Shopping Mall  0.05
4  Middle Eastern Restaurant  0.05


----Downsview Northwest----
                  venue  freq
0        Discount Store   0.2
1    Athletics & Sports   0.2
2  Gym / Fitness Center   0.2
3          Liquor Store   0.2
4         Grocery Store   0.2


----Downsview West----
           venue  freq
0  Grocery Store   0.4
1  Shopping Mall   0.2
2           Bank   0.2
3           Park   0.2
4    Yoga Studio   0.0


----Dufferin----
            venue  freq
0          Bakery  0.10
1        Pharmacy  0.10
2     Supermarket  0.10
3     Music Venue  0.05
4  Discount Store  0.05


----East Birchmount Park----
                             venue  freq
0                   Discount Store   0.4
1                 Department Store   0.2
2                      Coffee Shop   0.2
3               Chinese Restaurant   0.2
4  Molecular Gastr

               venue  freq
0        Pizza Place  0.25
1  Mobile Phone Shop  0.25
2               Park  0.25
3           Bus Line  0.25
4      Movie Theater  0.00


----Kingsway Park South East----
               venue  freq
0               Pool   0.5
1     Baseball Field   0.5
2        Yoga Studio   0.0
3  Mobile Phone Shop   0.0
4      Movie Theater   0.0


----Kingsway Park South West----
               venue  freq
0  Convenience Store  0.08
1      Burrito Place  0.08
2     Discount Store  0.08
3    Supplement Shop  0.08
4     Sandwich Place  0.08


----L'Amoreaux East----
                venue  freq
0          Playground  0.33
1                 Gym  0.33
2                Park  0.33
3         Yoga Studio  0.00
4  Miscellaneous Shop  0.00


----L'Amoreaux West----
                  venue  freq
0  Fast Food Restaurant  0.13
1    Chinese Restaurant  0.13
2           Coffee Shop  0.13
3           Pizza Place  0.07
4         Grocery Store  0.07


----Lawrence Heights----
                 

               venue  freq
0               Bank   1.0
1        Yoga Studio   0.0
2  Mobile Phone Shop   0.0
3      Movie Theater   0.0
4              Motel   0.0


----Railway Lands----
              venue  freq
0   Airport Service  0.19
1  Airport Terminal  0.12
2    Airport Lounge  0.12
3     Boat or Ferry  0.06
4       Coffee Shop  0.06


----Rathnelly----
              venue  freq
0       Coffee Shop  0.13
1               Pub  0.13
2      Liquor Store  0.07
3  Sushi Restaurant  0.07
4        Restaurant  0.07


----Regent Park----
         venue  freq
0  Coffee Shop  0.18
1          Pub  0.06
2         Park  0.06
3       Bakery  0.06
4         Café  0.06


----Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Richview Gardens----
               venue  freq
0        Pizza Place  0.25
1  Mobile Phone Shop  0.25
2               Park  0.25
3           Bus Line  0.25
4  

               venue  freq
0  Convenience Store  0.08
1      Burrito Place  0.08
2     Discount Store  0.08
3    Supplement Shop  0.08
4     Sandwich Place  0.08


----Thistletown----
                 venue  freq
0        Grocery Store  0.22
1          Pizza Place  0.11
2          Coffee Shop  0.11
3           Beer Store  0.11
4  Fried Chicken Joint  0.11


----Thorncliffe Park----
               venue  freq
0     Sandwich Place  0.11
1  Indian Restaurant  0.11
2        Yoga Studio  0.06
3      Grocery Store  0.06
4        Supermarket  0.06


----Toronto Dominion Centre----
                venue  freq
0         Coffee Shop  0.14
1                Café  0.08
2               Hotel  0.06
3          Restaurant  0.04
4  Italian Restaurant  0.04


----Toronto Islands----
                venue  freq
0         Coffee Shop  0.11
1            Aquarium  0.05
2               Hotel  0.05
3                Café  0.04
4  Italian Restaurant  0.04


----Trinity----
              venue  freq
0            

#### Putting it into Pandas

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,American Restaurant,Gym,Asian Restaurant,Hotel,Cosmetics Shop
1,Agincourt,Lounge,Breakfast Spot,Clothing Store,Skating Rink,Women's Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant
2,Agincourt North,Gym,Playground,Park,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
3,Albion Gardens,Grocery Store,Pharmacy,Coffee Shop,Sandwich Place,Beer Store,Fast Food Restaurant,Fried Chicken Joint,Pizza Place,Drugstore,Dumpling Restaurant
4,Alderwood,Pizza Place,Pharmacy,Skating Rink,Coffee Shop,Pool,Pub,Sandwich Place,Gym,Airport Service,Deli / Bodega


## Clustering Neighborhoods

In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 3, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Fast Food Restaurant,Bus Stop,Park,Food & Drink Shop,Women's Store,Doner Restaurant,Diner,Discount Store,Dive Bar,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Portuguese Restaurant,Pizza Place,Hockey Arena,Coffee Shop,Intersection,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Ethiopian Restaurant,Drugstore
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1.0,Coffee Shop,Pub,Bakery,Park,Café,Gym / Fitness Center,Mexican Restaurant,Breakfast Spot,Theater,Beer Store
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,1.0,Coffee Shop,Pub,Bakery,Park,Café,Gym / Fitness Center,Mexican Restaurant,Breakfast Spot,Theater,Beer Store
4,M6A,North York,Lawrence Heights,43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Women's Store,Sporting Goods Shop,Miscellaneous Shop,Boutique,Athletics & Sports,Gift Shop,Event Space,Accessories Store
5,M6A,North York,Lawrence Manor,43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Women's Store,Sporting Goods Shop,Miscellaneous Shop,Boutique,Athletics & Sports,Gift Shop,Event Space,Accessories Store
6,M7A,Queen's Park,Not assigned,43.662301,-79.389494,1.0,Coffee Shop,Gym,Park,Yoga Studio,Diner,Italian Restaurant,Japanese Restaurant,Fast Food Restaurant,Seafood Restaurant,Sandwich Place
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,M1B,Scarborough,Rouge,43.806686,-79.194353,1.0,Fast Food Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
9,M1B,Scarborough,Malvern,43.806686,-79.194353,1.0,Fast Food Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore


In [36]:
toronto_merged.dropna()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Fast Food Restaurant,Bus Stop,Park,Food & Drink Shop,Women's Store,Doner Restaurant,Diner,Discount Store,Dive Bar,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Portuguese Restaurant,Pizza Place,Hockey Arena,Coffee Shop,Intersection,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Ethiopian Restaurant,Drugstore
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1.0,Coffee Shop,Pub,Bakery,Park,Café,Gym / Fitness Center,Mexican Restaurant,Breakfast Spot,Theater,Beer Store
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,1.0,Coffee Shop,Pub,Bakery,Park,Café,Gym / Fitness Center,Mexican Restaurant,Breakfast Spot,Theater,Beer Store
4,M6A,North York,Lawrence Heights,43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Women's Store,Sporting Goods Shop,Miscellaneous Shop,Boutique,Athletics & Sports,Gift Shop,Event Space,Accessories Store
5,M6A,North York,Lawrence Manor,43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Women's Store,Sporting Goods Shop,Miscellaneous Shop,Boutique,Athletics & Sports,Gift Shop,Event Space,Accessories Store
6,M7A,Queen's Park,Not assigned,43.662301,-79.389494,1.0,Coffee Shop,Gym,Park,Yoga Studio,Diner,Italian Restaurant,Japanese Restaurant,Fast Food Restaurant,Seafood Restaurant,Sandwich Place
8,M1B,Scarborough,Rouge,43.806686,-79.194353,1.0,Fast Food Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
9,M1B,Scarborough,Malvern,43.806686,-79.194353,1.0,Fast Food Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
10,M3B,North York,Don Mills North,43.745906,-79.352188,1.0,Basketball Court,Baseball Field,Japanese Restaurant,Caribbean Restaurant,Café,Gym / Fitness Center,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore


In [33]:
toronto_merged=[toronto_merged]

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

ValueError: cannot convert float NaN to integer